# NVIDIA NIM TTS Model Deployment on Amazon SageMaker AI using BYOC (Bring Your Own Container)

## Introduction

This notebook demonstrates how to deploy the **NVIDIA NIM TTS (Magpie TTS Multilingual)** model for Text-to-Speech (TTS) tasks using Amazon SageMaker with a custom container that supports both HTTP and gRPC protocols.

### About NVIDIA NIM TTS (Magpie)

The **NVIDIA NIM TTS Magpie Multilingual** provides a production-ready text-to-speech service:

- **Architecture**: HTTP + gRPC routing to NVIDIA NIM TTS container
- **Model**: Magpie TTS Multilingual optimized for high-quality speech synthesis
- **Performance**: Low latency, high-quality audio output
- **Features**: Multiple voices, languages, zero-shot voice cloning, custom dictionaries
- **Deployment**: Ready for SageMaker real-time inference

### Key Features

1. **Dual Protocol Support**: HTTP for simple requests, gRPC for advanced features
2. **Multilingual Support**: Multiple languages and voices available
3. **Zero-Shot Voice Cloning**: Clone voices from audio prompts (gRPC)
4. **Custom Dictionaries**: Define custom pronunciations (gRPC)
5. **End-to-End Streaming**: True streaming via `InvokeEndpointWithResponseStream` API
6. **Production Ready**: Built with NVIDIA NIM for enterprise deployment

## Prerequisites and Setup

**❗ Important Notes:**
- Docker is required to pull and push container images
- You need an **NGC_API_KEY** from NVIDIA NGC ([Get one here](https://build.nvidia.com))
- ECR permissions are required for pushing Docker images to your private ECR
- NIM ECR image is currently available only in `us-east-1` region

**Supported AWS Instances (Compute Capability >= 8.0):**

| Instance Family | GPU | Examples |
|-----------------|-----|----------|
| ml.g6e.* | L40S | ml.g6e.xlarge, ml.g6e.2xlarge |
| ml.p4d.* | A100 | ml.p4d.24xlarge |
| ml.p5.* | H100 | ml.p5.48xlarge |

> ⚠️ Other GPU instances (g4dn, g5, p3) are **not supported**.

In [ ]:
# Install required packages
%pip install sagemaker>=2.246.0 boto3 soundfile --quiet

In [ ]:
# Import required packages
import boto3
import json
import sagemaker
import time
import os
from sagemaker import get_execution_role
from pathlib import Path

# Setup AWS clients and session
sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
sm_runtime = boto3.client("sagemaker-runtime")
region = sess.region_name
sts_client = sess.client('sts')
account_id = sts_client.get_caller_identity()['Account']

print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"Role: {role}")

In [ ]:
# Define deployment arguments
public_nim_image = "public.ecr.aws/nvidia/nim:magpie-tts-multilingual-1.6.0"
nim_model = "magpie-tts-multilingual"
sm_model_name = "nim-tts-magpie-multilingual"
instance_type = "ml.g6e.xlarge"

## NIM Container Setup

Pull the NIM image from public ECR and push to your private ECR repository:

In [ ]:
# Pull NIM image from public ECR and push to private ECR

import subprocess

print(f"Public NIM Image: {public_nim_image}")
print(f"Target model name: {nim_model}")

bash_script = f"""
echo "Public NIM Image: {public_nim_image}"
docker pull {public_nim_image}

echo "Resolved account: {account_id}"
echo "Resolved region: {region}"

nim_image="{account_id}.dkr.ecr.{region}.amazonaws.com/{nim_model}"

# Ensure the repository name adheres to AWS constraints
repository_name=$(echo "{nim_model}" | tr '[:upper:]' '[:lower:]' | tr -cd '[:alnum:]._/-')

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "$repository_name" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "$repository_name" > /dev/null
    echo "✅ Created ECR repository: $repository_name"
else
    echo "✅ ECR repository already exists: $repository_name"
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin "{account_id}.dkr.ecr.{region}.amazonaws.com"

docker tag {public_nim_image} $nim_image
docker push $nim_image
echo "✅ Image pushed successfully"
echo -n $nim_image
"""

nim_image = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{nim_model}"

# Run the bash script and capture real-time output
process = subprocess.Popen(bash_script, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

while True:
    output = process.stdout.readline()
    if output == b'' and process.poll() is not None:
        break
    if output:
        print(output.decode().strip())

stderr = process.stderr.read().decode()
if stderr:
    print("Errors:", stderr)

print(f"\n🎯 Private ECR Image: {nim_image}")

In [ ]:
# Print the private ECR NIM image that will be used for SageMaker deployment
print(f"NIM Image URI: {nim_image}")

## Create SageMaker Endpoint

**Before proceeding further, please set your NGC API Key.**

In [ ]:
# SET YOUR NGC API KEY HERE
# Required for running NIM - get yours from https://build.nvidia.com
NGC_API_KEY = None  # <-- SET ME

In [ ]:
# Validate NGC API Key
assert NGC_API_KEY is not None, "NGC API KEY is not set. Please set the NGC_API_KEY variable in the previous cell."
print("✅ NGC_API_KEY is set")

In [ ]:
# Create SageMaker model with NIM container
container = {
    "Image": nim_image,
    "Environment": {
        "NGC_API_KEY": NGC_API_KEY,
    }
}

create_model_response = sm.create_model(
    ModelName=sm_model_name, 
    ExecutionRoleArn=role, 
    PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

In [ ]:
# Create endpoint configuration
endpoint_config_name = sm_model_name

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
            "ContainerStartupHealthCheckTimeoutInSeconds": 1800,
            "InferenceAmiVersion": "al2-ami-sagemaker-inference-gpu-2"
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
# Create endpoint
endpoint_name = sm_model_name

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
# Wait for endpoint to be in service
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

## Inference Testing

Test the deployed TTS endpoint with different transport options.

### API Request Format

The request body follows the [NVIDIA Riva TTS SynthesizeSpeechRequest proto](https://docs.nvidia.com/nim/riva/tts/1.6.0/protos.html#nvidia-riva-tts-synthesizespeechrequest):

```json
{
    "text": "Text to synthesize",
    "voice_name": "Magpie-Multilingual.EN-US.Aria",
    "language_code": "en-US",
    "sample_rate_hz": 44100,
    "encoding": "LINEAR_PCM",
    "zero_shot_data": {
        "audio_prompt": "<base64-encoded-audio>",
        "quality": 20,
        "transcript": "optional transcript"
    },
    "custom_dictionary": "NVIDIA  en-vid-ee-ah,SageMaker  sage-may-ker"
}
```

| Field | Type | Description |
|-------|------|-------------|
| `text` | string | Text to synthesize (required) |
| `voice_name` | string | Voice name ([available voices](https://docs.nvidia.com/nim/riva/tts/1.10.0/support-matrix.html#available-voices)) |
| `language_code` | string | Language code: en-US, es-US, fr-FR, de-DE, zh-CN, vi-VN, it-IT ([docs](https://docs.nvidia.com/nim/riva/tts/1.10.0/support-matrix.html#magpie-tts-multilingual)) |
| `sample_rate_hz` | int | Sample rate (default: 44100) |
| `encoding` | string | `LINEAR_PCM` or `OGGOPUS` |
| `zero_shot_data` | object | `{audio_prompt, quality, transcript}` for voice cloning |
| `custom_dictionary` | string | `"word1  pron1,word2  pron2"` (double-space separator) |

**Response:** JSON matching [NIM SynthesizeSpeechResponse](https://docs.nvidia.com/nim/riva/tts/1.6.0/protos.html#nvidia-riva-tts-synthesizespeechresponse):

```json
{
    "audio": "<base64-encoded-audio>",
    "meta": {
        "text": "original input text",
        "processed_text": "text after preprocessing",
        "predicted_durations": [0.1, 0.2, ...]
    }
}
```

| Response Field | Type | Description |
|----------------|------|-------------|
| `audio` | string | Base64-encoded audio bytes (always present) |
| `meta` | object | Optional - metadata from NIM (if returned) |

**Streaming:** Use `CustomAttributes='/invocations/stream'` header with `invoke_endpoint_with_response_stream`

### Transport Selection Options

The TTS endpoint supports both HTTP and gRPC protocols internally:

| Transport | Use Case | Features |
|-----------|----------|----------|
| **HTTP** (default) | Simple TTS requests | Basic synthesis, voice selection |
| **gRPC** | Advanced features | Zero-shot cloning, streaming, custom dictionaries |
| **Auto** | Automatic selection | HTTP for simple, gRPC for advanced features |

### Transport Selection (via CustomAttributes Header)

Use the `CustomAttributes` header to select transport:
- `/invocations/http` - Force HTTP transport
- `/invocations/grpc` - Force gRPC transport  
- `/invocations/stream` - Enable streaming (uses gRPC)

If not specified, auto-routing selects HTTP for simple requests, gRPC for advanced features.

In [ ]:
import IPython.display as ipd
import base64

def test_tts(text, voice_name=None, language_code="en-US", sample_rate_hz=44100, 
             encoding="LINEAR_PCM", zero_shot_data=None, custom_dictionary=None,
             custom_attributes=None, output_file="tts_output.wav"):
    """
    Test TTS endpoint following NIM SynthesizeSpeechRequest proto format.
    
    Note: For streaming, use test_tts_streaming() with invoke_endpoint_with_response_stream.
    
    Args:
        text: Text to synthesize (required)
        voice_name: Voice name (NIM standard field)
        language_code: Language code (default: en-US)
        sample_rate_hz: Sample rate (default: 44100)
        encoding: Audio encoding - LINEAR_PCM or OGGOPUS
        zero_shot_data: Dict with {audio_prompt, quality, transcript} for voice cloning
        custom_dictionary: NIM format string "word1  pronunciation1,word2  pronunciation2"
        custom_attributes: SageMaker CustomAttributes header for transport selection
        output_file: Output filename for the audio
    """
    print(f"Testing TTS with text: '{text[:50]}...'")
    
    # Build payload following NIM SynthesizeSpeechRequest proto
    payload = {
        "text": text,
        "language_code": language_code,
        "sample_rate_hz": sample_rate_hz,
        "encoding": encoding
    }
    
    if voice_name:
        payload["voice_name"] = voice_name
    if zero_shot_data:
        payload["zero_shot_data"] = zero_shot_data
    if custom_dictionary:
        payload["custom_dictionary"] = custom_dictionary
    
    try:
        # Build invoke_endpoint kwargs
        invoke_kwargs = {
            "EndpointName": endpoint_name,
            "ContentType": "application/json",
            "Body": json.dumps(payload)
        }
        
        # Add custom attributes if specified (for transport selection)
        if custom_attributes:
            invoke_kwargs["CustomAttributes"] = custom_attributes
        
        response = sm_runtime.invoke_endpoint(**invoke_kwargs)
        
        # Response is JSON matching NIM SynthesizeSpeechResponse proto
        # https://docs.nvidia.com/nim/riva/tts/1.6.0/protos.html#nvidia-riva-tts-synthesizespeechresponse
        response_body = response['Body'].read()
        result = json.loads(response_body)
        
        print(f"\n✅ TTS inference successful!")
        
        # Extract audio from response (base64 encoded)
        audio_bytes = base64.b64decode(result['audio'])
        print(f"Audio size: {len(audio_bytes):,} bytes")
        
        # Print metadata if available
        if 'meta' in result:
            print(f"Metadata: {result['meta']}")
        
        # Save audio to WAV file
        with open(output_file, 'wb') as f:
            f.write(audio_bytes)
        print(f"Audio saved to: {output_file}")
        
        # Play audio in notebook
        return ipd.Audio(output_file)
        
    except Exception as e:
        print(f"❌ TTS test failed: {e}")
        return None

### Test 1: Auto Transport (HTTP for simple requests)

Uses HTTP by default for simple requests, automatically switches to gRPC for advanced features:

In [ ]:
# Test 1: Auto transport (HTTP for simple requests)
# Simple requests use HTTP by default
test_tts(
    text="Hello! This is a test using automatic transport selection.",
    output_file="tts_auto.wav"
)

### Test 2: Force HTTP Transport via Custom Attributes Header

Use the `CustomAttributes` parameter in `invoke_endpoint` to force HTTP transport:

In [ ]:
# Test 2: Force HTTP transport using CustomAttributes header
# This is the recommended SageMaker-native way to select transport
test_tts(
    text="This audio is generated using HTTP transport via custom attributes header.",
    custom_attributes="/invocations/http",
    output_file="tts_http_header.wav"
)

### Test 3: Force gRPC Transport via Custom Attributes Header

Use gRPC for advanced features like streaming, zero-shot voice cloning, and custom dictionaries:

In [ ]:
# Test 3: Force gRPC transport using CustomAttributes header
# Use this when you need advanced features
test_tts(
    text="This audio is generated using gRPC transport via custom attributes header.",
    custom_attributes="/invocations/grpc",
    output_file="tts_grpc_header.wav"
)

### Test 4: Different Voice Selection

Test with a specific voice name. Voice names follow the pattern: `Magpie-Multilingual.LANGUAGE.VoiceName`

For a complete list of available voices, see the [NVIDIA NIM TTS Support Matrix](https://docs.nvidia.com/nim/riva/tts/1.10.0/support-matrix.html#available-voices).

**Supported Languages:** English (en-US), Spanish (es-US), French (fr-FR), German (de-DE), Mandarin (zh-CN), Vietnamese (vi-VN), Italian (it-IT)

**Example voices:**
- `Magpie-Multilingual.EN-US.Aria` - English female
- `Magpie-Multilingual.EN-US.Jason` - English male
- `Magpie-Multilingual.ES-US.Diego` - Spanish male
- `Magpie-Multilingual.FR-FR.Pascal` - French male

In [ ]:
# Test 4: Use a specific voice
# Voice names follow the pattern: Magpie-Multilingual.LANGUAGE.VoiceName
test_tts(
    text="This audio demonstrates using a specific voice for synthesis.",
    voice_name="Magpie-Multilingual.EN-US.Aria",
    output_file="tts_voice.wav"
)

In [ ]:
# Test 4b: Different sample rate
# Try a lower sample rate (22050 Hz)
test_tts(
    text="This audio is generated with a lower sample rate for smaller file size.",
    sample_rate_hz=22050,
    output_file="tts_low_sample_rate.wav"
)

### Test 5: Force gRPC with Longer Text

When processing longer texts, gRPC is recommended for better performance. Use `custom_attributes="/invocations/grpc"` to force gRPC transport:

In [ ]:
# Test 5: Force gRPC for longer text
# Use custom_attributes="/invocations/grpc" for longer texts or gRPC-specific features
test_tts(
    text="This is a longer text that demonstrates gRPC transport. "
         "gRPC is recommended for generating audio for longer texts "
         "because it provides better performance and reliability.",
    custom_attributes="/invocations/grpc",  # Force gRPC transport
    output_file="tts_grpc_long.wav"
)

### Test 6: Custom Dictionary (gRPC-only)

Custom dictionaries allow you to define custom pronunciations for specific words:

In [ ]:
# Test 6: Custom dictionary (gRPC-only feature)
# NIM format: comma-separated key-value pairs with double-space separator
# This automatically triggers gRPC in "auto" mode

test_tts(
    text="Welcome to NVIDIA and Amazon SageMaker integration.",
    custom_dictionary="NVIDIA  en-vid-ee-ah,SageMaker  sage-may-ker",  # NIM format
    output_file="tts_custom_dict.wav"
)

### Test 7: Streaming with Audio Output

Use SageMaker's `invoke_endpoint_with_response_stream` API for true end-to-end streaming with lower time-to-first-audio. This test shows timing metrics and outputs a playable audio file:

In [ ]:
# Test 7: End-to-End Streaming with Audio Output
# Collects streaming chunks, shows timing info, and outputs playable audio

import wave
import io
import time

def pcm_to_wav(pcm_data, sample_rate=44100, channels=1, bits_per_sample=16):
    """Convert raw PCM audio data to WAV format."""
    buffer = io.BytesIO()
    with wave.open(buffer, 'wb') as wav_file:
        wav_file.setnchannels(channels)
        wav_file.setsampwidth(bits_per_sample // 8)
        wav_file.setframerate(sample_rate)
        wav_file.writeframesraw(pcm_data)
    return buffer.getvalue()

def test_tts_streaming(text, voice_name=None, language_code="en-US", sample_rate_hz=44100, 
                       encoding="LINEAR_PCM", output_file="tts_streaming.wav"):
    """
    Test TTS with end-to-end streaming using InvokeEndpointWithResponseStream.
    
    Shows timing metrics and outputs a playable audio file.
    """
    print(f"🎤 Streaming TTS Test")
    print(f"Text: '{text[:60]}...'" if len(text) > 60 else f"Text: '{text}'")
    print("-" * 50)
    
    payload = {
        "text": text,
        "language_code": language_code,
        "sample_rate_hz": sample_rate_hz,
        "encoding": encoding
    }
    if voice_name:
        payload["voice_name"] = voice_name
    
    try:
        response = sm_runtime.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            ContentType='application/json',
            CustomAttributes='/invocations/stream',
            Body=json.dumps(payload)
        )
        
        start_time = time.time()
        first_chunk_time = None
        chunks = []
        
        for event in response['Body']:
            if 'PayloadPart' in event:
                chunk = event['PayloadPart']['Bytes']
                if chunk:
                    if first_chunk_time is None:
                        first_chunk_time = time.time() - start_time
                    chunks.append(chunk)
        
        total_time = time.time() - start_time
        raw_pcm = b''.join(chunks)
        
        print(f"⏱️  Time to first chunk: {first_chunk_time:.3f}s")
        print(f"📦 Chunks received: {len(chunks)}")
        print(f"📊 Total PCM bytes: {len(raw_pcm):,}")
        print(f"⏱️  Total time: {total_time:.3f}s")
        
        # Convert to WAV and save
        wav_data = pcm_to_wav(raw_pcm, sample_rate=sample_rate_hz)
        with open(output_file, 'wb') as f:
            f.write(wav_data)
        print(f"\n✅ Audio saved to: {output_file}")
        
        return ipd.Audio(output_file)
        
    except Exception as e:
        print(f"❌ Streaming failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run streaming test with audio output
test_tts_streaming(
    text="Welcome to NVIDIA's text-to-speech streaming demonstration. "
         "This test shows how audio chunks are delivered incrementally, "
         "reducing time to first audio for real-time applications.",
    output_file="tts_streaming.wav"
)

### Test 8: Live Streaming Demo (Watch Chunks Arrive)

This demonstrates real-time streaming - watch the audio chunks arrive live as they're generated by the model:

In [ ]:
# Live Streaming Demo - Watch chunks arrive in real-time like chat streaming
# Each chunk is printed as it arrives from the model

from IPython.display import display, clear_output
import sys

text = ("This is a live streaming demonstration. Watch as each audio chunk arrives "
        "from the NVIDIA NIM TTS model in real-time. This is similar to how chat "
        "applications stream text tokens, but here we're streaming audio data.")

print("🎤 LIVE STREAMING DEMO")
print(f"Text: '{text[:50]}...'")
print("=" * 70)
print()

payload = {
    "text": text,
    "language_code": "en-US",
    "sample_rate_hz": 44100,
    "encoding": "LINEAR_PCM"
}

try:
    response = sm_runtime.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name,
        ContentType='application/json',
        CustomAttributes='/invocations/stream',
        Body=json.dumps(payload)
    )
    
    start_time = time.time()
    first_chunk_time = None
    chunk_count = 0
    total_bytes = 0
    
    for event in response['Body']:
        if 'PayloadPart' in event:
            chunk = event['PayloadPart']['Bytes']
            if chunk:
                chunk_count += 1
                total_bytes += len(chunk)
                
                if first_chunk_time is None:
                    first_chunk_time = time.time() - start_time
                    print(f"⏱️  First chunk arrived in {first_chunk_time:.3f}s!")
                    print("-" * 70)
                
                elapsed = time.time() - start_time
                # Print each chunk as it arrives - live streaming effect
                preview = chunk[:16].hex()
                print(f"📦 Chunk {chunk_count:3d} | {len(chunk):6,} bytes | Total: {total_bytes:8,} | @{elapsed:.2f}s | {preview}...")
                sys.stdout.flush()  # Force immediate output
    
    total_time = time.time() - start_time
    print("-" * 70)
    print(f"\n✅ Stream complete!")
    print(f"   Total chunks: {chunk_count}")
    print(f"   Total bytes: {total_bytes:,}")
    print(f"   Total time: {total_time:.3f}s")

except Exception as e:
    print(f"❌ Streaming failed: {e}")

### Transport Selection Summary

| CustomAttributes Value | Description |
|------------------------|-------------|
| `/invocations/http` | Force HTTP transport |
| `/invocations/grpc` | Force gRPC transport |
| `/invocations/stream` | Enable streaming (uses gRPC) |
| *(not set)* | Auto-routing: HTTP for simple, gRPC for advanced |

**gRPC-only features** (auto-trigger gRPC in auto mode):
- `zero_shot_data` - Voice cloning
- `custom_dictionary` - Custom pronunciations

### End-to-End Streaming

For true end-to-end streaming:
1. Use `CustomAttributes='/invocations/stream'` header
2. Use SageMaker's `invoke_endpoint_with_response_stream` API (NOT regular `invoke_endpoint`)
3. Streaming always uses gRPC internally

```python
# NIM SynthesizeSpeechRequest format (no stream field needed)
payload = {
    "text": "Your text to synthesize",
    "language_code": "en-US",
    "sample_rate_hz": 44100,
    "encoding": "LINEAR_PCM"
}

# Use invoke_endpoint_with_response_stream with CustomAttributes
response = sm_runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    CustomAttributes='/invocations/stream',
    Body=json.dumps(payload)
)
```

Streaming provides:
- Lower time-to-first-audio (audio starts playing before full synthesis completes)
- No message size limits (each chunk is small)
- Better user experience for long texts
- Returns raw PCM audio that needs WAV header conversion (see test_tts_streaming function)

## Resource Cleanup

**⚠️ Cost Warning**: Make sure to clean up resources when done testing.

In [ ]:
# Cleanup: Delete model, endpoint config, and endpoint
sm.delete_model(ModelName=sm_model_name)
print(f"✅ Model {sm_model_name} deleted")

sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
print(f"✅ Endpoint config {endpoint_config_name} deleted")

sm.delete_endpoint(EndpointName=endpoint_name)
print(f"✅ Endpoint {endpoint_name} deleted")

# Clean up generated audio files
import glob
audio_files = glob.glob("tts_*.wav")
for f in audio_files:
    try:
        os.remove(f)
        print(f"✅ Removed {f}")
    except Exception as e:
        print(f"⚠️ Could not remove {f}: {e}")